In [56]:
import os
import openai
import json
import tiktoken
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sqlalchemy import create_engine, Column, Integer, String, Text
from sqlalchemy.orm import declarative_base, sessionmaker

In [63]:
os.environ["OPENAI_API_KEY"] = "개인키입력"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [58]:
def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

# PDF 파일 로드 및 분할
loader = PyPDFLoader("D:/[24]ICT_Practice/practice_file/(2024)포트미스_가이드북.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, length_function=tiktoken_len)
texts = text_splitter.split_documents(pages)

# 임베딩 설정
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

docsearch = Chroma.from_documents(texts, hf)

# OpenAI API 설정
openai = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    streaming=False,
    temperature=0
)

qa = RetrievalQA.from_chain_type(
    llm=openai,
    chain_type="stuff",
    retriever=docsearch.as_retriever(
        search_type="mmr",
        search_kwargs={'k': 3, 'fetch_k': 10}
    ),
    return_source_documents=True
)

# SQLAlchemy 설정
DATABASE_URL = "sqlite:///conversations.db"
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# 대화 내용을 저장할 테이블 모델 정의
class Conversation(Base):
    __tablename__ = "conversations"

    id = Column(Integer, primary_key=True, index=True)
    user_id = Column(String, index=True)
    query = Column(String, index=True)
    response = Column(Text)

# 테이블 생성
Base.metadata.create_all(bind=engine)

# 대화 저장 함수
def save_conversation(db_session, user_id, query, response):
    conversation = Conversation(user_id=user_id, query=query, response=response)
    db_session.add(conversation)
    db_session.commit()

# 대화 조회 함수
def get_response(input_text):
    return qa.invoke({"query": input_text})

def print_all_conversations():
    db_session = SessionLocal()
    conversations = db_session.query(Conversation).all()
    for conv in conversations:
        print(f"ID: {conv.id}, User ID: {conv.user_id}, Query: {conv.query}, Response: {conv.response}")
    db_session.close()

def get_conversation_by_id(conversation_id):
    db_session = SessionLocal()
    conversation = db_session.query(Conversation).filter(Conversation.id == conversation_id).first()
    db_session.close()
    if conversation:
        return {
            "id": conversation.id,
            "user_id": conversation.user_id,
            "query": conversation.query,
            "response": conversation.response
        }
    else:
        return None

def get_conversations_by_user_id(user_id):
    db_session = SessionLocal()
    conversations = db_session.query(Conversation).filter(Conversation.user_id == user_id).all()
    db_session.close()
    return conversations

# 주피터 노트북용 함수
def save_query_response(user_id, input_text):
    try:
        chat_response = get_response(input_text)
        response_text = chat_response["result"]

        # 대화 내용 저장
        db_session = SessionLocal()
        save_conversation(db_session, user_id, input_text, response_text)
        db_session.close()

        output = {
            "status": 200,
            "message": "채팅 응답 성공",
            "body": {
                "chatMessage": response_text
            }
        }
    except Exception as e:
        output = {
            "status": 404,
            "message": "채팅 응답 실패",
            "body": {
                "error": str(e)
            }
        }

    return json.dumps(output, ensure_ascii=False, indent=4)

def show_all_conversations():
    print_all_conversations()

def show_conversation_by_id(conversation_id):
    conversation = get_conversation_by_id(conversation_id)
    if conversation:
        return f"ID: {conversation['id']}, User ID: {conversation['user_id']}, Query: {conversation['query']}, Response: {conversation['response']}"
    else:
        return "Conversation not found."

def show_conversations_by_user_id(user_id):
    conversations = get_conversations_by_user_id(user_id)
    if conversations:
        for conv in conversations:
            print(f"ID: {conv.id}, Query: {conv.query}, Response: {conv.response}")
    else:
        print("No conversations found for this user.")

In [59]:
#### 예시

In [60]:
user_id = "user123"

In [61]:
# 여러 질문 예시
input_text_1 = "선박입,출항신고 근거 법령이 뭐야?"
print(save_query_response(user_id, input_text_1))

input_text_2 = "선박의 종류에는 어떤 것들이 있지?"
print(save_query_response(user_id, input_text_2))

input_text_3 = "선박 검사의 절차는 어떻게 되지?"
print(save_query_response(user_id, input_text_3))


{
    "status": 404,
    "message": "채팅 응답 실패",
    "body": {
        "error": "(sqlite3.OperationalError) table conversations has no column named user_id\n[SQL: INSERT INTO conversations (user_id, \"query\", response) VALUES (?, ?, ?)]\n[parameters: ('user123', '선박입,출항신고 근거 법령이 뭐야?', '외항입항신고 및 승무원/승객명부 작성에 대한 근거 법령은 \"선박입출항법 제4조 제1항, 시행령 제4조, 시행규칙 제3조 제2항\"입니다.')]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)"
    }
}
{
    "status": 404,
    "message": "채팅 응답 실패",
    "body": {
        "error": "(sqlite3.OperationalError) table conversations has no column named user_id\n[SQL: INSERT INTO conversations (user_id, \"query\", response) VALUES (?, ?, ?)]\n[parameters: ('user123', '선박의 종류에는 어떤 것들이 있지?', '컨테이너선, 세미(혼재)컨테이너선, 자동차운반선, LNG운반선, 원유운반선, 석유제품운반선, 케미칼운반선이 있습니다.')]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)"
    }
}
{
    "status": 404,
    "message": "채팅 응답 실패",
    "body": {
        "error": "(sqlite3.OperationalError) table conversations has no 

In [62]:
# 전체 기록 조회
show_all_conversations()


OperationalError: (sqlite3.OperationalError) no such column: conversations.user_id
[SQL: SELECT conversations.id AS conversations_id, conversations.user_id AS conversations_user_id, conversations."query" AS conversations_query, conversations.response AS conversations_response 
FROM conversations]
(Background on this error at: https://sqlalche.me/e/20/e3q8)